## 数据增强

In [7]:
import os
import random

from PIL import Image

def place_images_on_large_canvas(images, canvas_size=(1280, 1280), min_gap=2, max_gap=5):
    # 初始化一个空的画布
    canvas_width, canvas_height = canvas_size
    canvas = Image.new('RGB', canvas_size, (255, 255, 255))  # 创建一个白色背景的画布
    x_offset, y_offset = 0, 0  # 当前插入小图的坐标
    # 结果图列表
    all_canvases = [canvas]  # 用来存储大图列表
    # 遍历所有小图
    for image in images:
        image_width, image_height = image.size
        # 随机生成间距
        gap = random.randint(min_gap, max_gap)
        # 如果当前小图放不下，创建新的大图并重置坐标
        if x_offset + image_width + gap > canvas_width:  # 换行
            x_offset = 0
            y_offset += image_height + gap
            if y_offset + image_height + gap > canvas_height:  # 换到新的大图
                new_canvas = Image.new('RGB', canvas_size, (255, 255, 255))
                all_canvases.append(new_canvas)  # 添加新的大图
                canvas = new_canvas
                y_offset = 0  # 重置y坐标
        if y_offset + image_height + gap > canvas_height:
            continue  # 当前图无法放下，跳过

        # 将小图粘贴到当前画布上
        canvas.paste(image, (x_offset, y_offset))
        x_offset += image_width + gap  # 更新x坐标，考虑间距

    return all_canvases

# 示例使用
if __name__ == "__main__":
    # 假设images是包含所有小图的列表
    source_images_dir = "./source"
    images = [Image.open(os.path.join(source_images_dir,file_path)) for file_path in os.listdir(source_images_dir)]

    canvases = place_images_on_large_canvas(images)

    # 保存所有生成的图像
    for idx, canvas in enumerate(canvases):
        canvas.save(f"output/output_{idx}.png")
